In [1]:
import pandas as pd
import glob
import numpy as np
import scipy.sparse as ss
from collections import Counter
import time
#import pyranges as pr
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,collect_list, concat, lit, when, monotonically_increasing_id, concat_ws, broadcast
import pyspark.sql.functions as F
spark = SparkSession.builder.master("local").appName("parquet").getOrCreate()


'''
df = spark.read.parquet('/sbgenomics/project-files/BS_0302Y3N5.parquet/')
df = df.select(['contigName','start', 'end', 'names', 'referenceAllele', \
                 'alternateAlleles', 'qual', 'INFO_DP', 'splitFromMultiAllelic','genotypes'])
#print(df.count())

df = df.select(col('contigName').alias('chromosome'),'start','end',col('referenceAllele').alias('reference'),
         col('alternateAlleles').alias('alternate'),'qual','INFO_DP', 'splitFromMultiAllelic','genotypes')
df = df.withColumn('alternate',F.explode('alternate'))
df.show(1)'''

22/07/20 12:06:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


"\ndf = spark.read.parquet('/sbgenomics/project-files/BS_0302Y3N5.parquet/')\ndf = df.select(['contigName','start', 'end', 'names', 'referenceAllele',                  'alternateAlleles', 'qual', 'INFO_DP', 'splitFromMultiAllelic','genotypes'])\n#print(df.count())\n\ndf = df.select(col('contigName').alias('chromosome'),'start','end',col('referenceAllele').alias('reference'),\n         col('alternateAlleles').alias('alternate'),'qual','INFO_DP', 'splitFromMultiAllelic','genotypes')\ndf = df.withColumn('alternate',F.explode('alternate'))\ndf.show(1)"

In [2]:

r = spark.read.parquet('/sbgenomics/project-files/BS_0302Y3N5_chd_scored_variants.parquet')
print(r.count())
r.show(2)

177354
+---------+---------+---------+---------+-----------------+------+-------+---------------------+--------------------+-------+---------+----------+------------+----------+
|    start|      end|reference|alternate|unique_variant_id|  qual|INFO_DP|splitFromMultiAllelic|           genotypes| symbol|     rsID|cadd_score|eqtl_boolean|chromosome|
+---------+---------+---------+---------+-----------------+------+-------+---------------------+--------------------+-------+---------+----------+------------+----------+
|127508781|127508781|        T|        C|  6:127508781:T:C|558.77|     41|                 true|[{BS_0302Y3N5, 99...|YWHAZP4| rs553228|     0.071|           1|         6|
|149889587|149889587|        C|        T|  6:149889587:C:T|545.77|     37|                 true|[{BS_0302Y3N5, 99...| RAET1E|rs6925151|     3.341|           1|         6|
+---------+---------+---------+---------+-----------------+------+-------+---------------------+--------------------+-------+---------+---

In [3]:
# take just the variants w/ cadd scores strictly over 20.
gt20_vars = r.drop('unique_variant_id','qual','splitFromMultiAllelic').where((col('eqtl_boolean')!= 1))# &\
                                                                           # (col('cadd_score') < 20.0))
print(gt20_vars.count())
gt20_vars.show(5)

4822
+---------+---------+---------+---------+-------+--------------------+------+----+----------+------------+----------+
|    start|      end|reference|alternate|INFO_DP|           genotypes|symbol|rsID|cadd_score|eqtl_boolean|chromosome|
+---------+---------+---------+---------+-------+--------------------+------+----+----------+------------+----------+
|129486504|129486504|        C|        A|     46|[{BS_0302Y3N5, 99...|      |    |      22.2|           0|         6|
| 32581807| 32581807|        C|       CT|     55|[{BS_0302Y3N5, 99...|      |    |      23.5|           0|         6|
| 63280743| 63280743|        A|        G|     36|[{BS_0302Y3N5, 99...|      |    |      20.5|           0|         6|
| 32581793| 32581793|        T|        G|     52|[{BS_0302Y3N5, 18...|      |    |      24.1|           0|         6|
|129287979|129287979|        A|        C|     41|[{BS_0302Y3N5, 99...|      |    |      23.1|           0|         6|
+---------+---------+---------+---------+-------+--

In [21]:
r.drop('unique_variant_id','qual','splitFromMultiAllelic').where(col('symbol')!= '').count()

413349

In [2]:
df_join = spark.read.parquet('/sbgenomics/project-files/df_bed_leftsemi.parquet')
df_join.show(1)

+--------+--------+---------+---------+----+-------+---------------------+--------------------+--------------------+----------+
|   start|     end|reference|alternate|qual|INFO_DP|splitFromMultiAllelic|           genotypes|   unique_variant_id|chromosome|
+--------+--------+---------+---------+----+-------+---------------------+--------------------+--------------------+----------+
|35837428|35837431|        T|<NON_REF>|null|   null|                false|[{BS_0302Y3N5, 99...|14:35837428:T:<NO...|        14|
+--------+--------+---------+---------+----+-------+---------------------+--------------------+--------------------+----------+
only showing top 1 row



## Load in all additional files

In [5]:
cadd_scores_gt20 = spark.read.parquet('/sbgenomics/project-files/cadd_gt20_snps.parquet')
#cadd_scores_gt20.show(1)

In [7]:
cadd_chrom = spark.read.option("header","true").parquet('/sbgenomics/project-files/cadd_chrom_files_parquet/chr11')

cadd_chrom = cadd_chrom.select(col("0").alias("chromosome"), col("1").alias("start"),
                col("2").alias("reference"),col("3").alias("alternate"),col('5').alias('cadd_score'))
cadd_chrom = cadd_chrom.withColumn('unique_variant_id',concat(col("chromosome"),
                    lit(":"),col("start"),lit(':'),col('reference'),lit(':'),col('alternate')))
cadd_chrom.show(1)

+----------+------+---------+---------+----------+-----------------+
|chromosome| start|reference|alternate|cadd_score|unique_variant_id|
+----------+------+---------+---------+----------+-----------------+
|        11|426388|        A|        G|     3.628|    11:426388:A:G|
+----------+------+---------+---------+----------+-----------------+
only showing top 1 row



In [8]:
cadd_ALL_indels = spark.read.parquet('/sbgenomics/project-files/cadd_gnomad_all.parquet/')
cadd_ALL_indels.show(1)

+--------+---------+---------+----------+--------------------+----------+
|   start|reference|alternate|cadd_score|   unique_variant_id|chromosome|
+--------+---------+---------+----------+--------------------+----------+
|36449839|        A|AAAAAAAAC|     0.513|1:36449839:A:AAAA...|         1|
+--------+---------+---------+----------+--------------------+----------+
only showing top 1 row



In [7]:
cadd_indels = spark.read.parquet('/sbgenomics/project-files/cadd_gt20_gnomAD_all_chroms.parquet')
cadd_indels.show(1)

+-----+---------+---------+----------+----------+-----------------+
|start|reference|alternate|cadd_score|chromosome|unique_variant_id|
+-----+---------+---------+----------+----------+-----------------+
|47070|        T|       TC|      23.2|        10|    10:47070:T:TC|
+-----+---------+---------+----------+----------+-----------------+
only showing top 1 row



In [10]:
eqtls_high_impact = spark.read.parquet('/sbgenomics/project-files/eqtls_and_high_impact.parquet')
eqtls_high_impact.show(1)

+------+---------+---------+-----------+------+-----------------+----------+
| start|reference|alternate|       rsID|symbol|unique_variant_id|chromosome|
+------+---------+---------+-----------+------+-----------------+----------+
|668472|        C|        A|rs372034137|ZNF595|     4:668472:C:A|         4|
+------+---------+---------+-----------+------+-----------------+----------+
only showing top 1 row



In [11]:
whole_genome_bed = spark.read.option("header","true").parquet(
                            '/sbgenomics/project-files/bed_protein_coding_coords_only.parquet')
whole_genome_bed.show(1)

+---------+--------+--------------+
|start_bed| end_bed|chromosome_bed|
+---------+--------+--------------+
| 38297023|38323217|            17|
+---------+--------+--------------+
only showing top 1 row



## Begin workflow

In [3]:
df_NonRef = df_join.where( (col('alternate') == '<NON_REF>')) # will add this back on later!
df_noNonRef = df_join.where(~ (col('alternate') == '<NON_REF>')) # only need to get CADD scores for this df

df_snps = df_noNonRef.where((F.length("reference") == 1) & (F.length("alternate") == 1))                    
df_indels = df_noNonRef.where(~ ((F.length("reference") == 1) & (F.length("alternate") == 1)))

In [10]:
result_snps = df_snps.join(cadd_scores_gt20.select('unique_variant_id','cadd_score'), ["unique_variant_id"], "leftsemi")
print(result_snps.count())
result_snps.show(1)

NameError: name 'cadd_scores_gt20' is not defined

2785

In [9]:
result_indels = df_indels.join(cadd_indels.select('unique_variant_id','cadd_score'), ["unique_variant_id"], "inner");
print(result_indels.count())
result_indels.show(1)

3


+-----------------+--------+--------+---------+---------+------+-------+---------------------+--------------------+----------+----------+
|unique_variant_id|   start|     end|reference|alternate|  qual|INFO_DP|splitFromMultiAllelic|           genotypes|chromosome|cadd_score|
+-----------------+--------+--------+---------+---------+------+-------+---------------------+--------------------+----------+----------+
|  3:27717830:T:TA|27717830|27717831|        T|       TA|289.73|     29|                 true|[{BS_0302Y3N5, 99...|         3|      21.7|
+-----------------+--------+--------+---------+---------+------+-------+---------------------+--------------------+----------+----------+
only showing top 1 row



In [25]:
result_snps = result_snps.select(result_snps.columns+[lit('').alias("symbol"),lit('').alias("rsID")])
result_indels = result_indels.select(result_indels.columns+[lit('').alias("symbol"),lit('').alias("rsID")])

results_gt20 = result_snps.union(result_indels)
print(results_gt20.count())
results_gt20.show(1)

22/06/28 12:14:27 ERROR BroadcastExchangeExec: Could not execute broadcast in 300 secs.
java.util.concurrent.TimeoutException
	at java.util.concurrent.FutureTask.get(FutureTask.java:205)
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.doExecuteBroadcast(BroadcastExchangeExec.scala:194)
	at org.apache.spark.sql.execution.InputAdapter.doExecuteBroadcast(WholeStageCodegenExec.scala:515)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeBroadcast$1(SparkPlan.scala:193)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org.apache.spark.sql.execution.SparkPlan.executeBroadcast(SparkPlan.scala:189)
	at org.apache.spark.sql.execution.joins.BroadcastHashJoinExec.prepareBroadcast(BroadcastHashJoinExec.scala:203)
	at org.apache.spark.sql.execution.joins.BroadcastHas

Py4JJavaError: An error occurred while calling o287.count.
: org.apache.spark.SparkException: Could not execute broadcast in 300 secs. You can increase the timeout for broadcasts via spark.sql.broadcastTimeout or disable broadcast join by setting spark.sql.autoBroadcastJoinThreshold to -1
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.doExecuteBroadcast(BroadcastExchangeExec.scala:205)
	at org.apache.spark.sql.execution.InputAdapter.doExecuteBroadcast(WholeStageCodegenExec.scala:515)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeBroadcast$1(SparkPlan.scala:193)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org.apache.spark.sql.execution.SparkPlan.executeBroadcast(SparkPlan.scala:189)
	at org.apache.spark.sql.execution.joins.BroadcastHashJoinExec.

In [29]:
result_snps.show(1)

+-----------------+--------+--------+---------+---------+----+-------+---------------------+--------------------+----------+------+----+------+----+
|unique_variant_id|   start|     end|reference|alternate|qual|INFO_DP|splitFromMultiAllelic|           genotypes|chromosome|symbol|rsID|symbol|rsID|
+-----------------+--------+--------+---------+---------+----+-------+---------------------+--------------------+----------+------+----+------+----+
|  10:27908546:T:G|27908546|27908547|        T|        G| 0.0|     50|                 true|[{BS_0302Y3N5, 99...|        10|      |    |      |    |
+-----------------+--------+--------+---------+---------+----+-------+---------------------+--------------------+----------+------+----+------+----+
only showing top 1 row



In [30]:
result_indels.show(1)

+-----------------+--------+--------+---------+---------+------+-------+---------------------+--------------------+----------+------+----+------+----+
|unique_variant_id|   start|     end|reference|alternate|  qual|INFO_DP|splitFromMultiAllelic|           genotypes|chromosome|symbol|rsID|symbol|rsID|
+-----------------+--------+--------+---------+---------+------+-------+---------------------+--------------------+----------+------+----+------+----+
|  3:27717830:T:TA|27717830|27717831|        T|       TA|289.73|     29|                 true|[{BS_0302Y3N5, 99...|         3|      |    |      |    |
+-----------------+--------+--------+---------+---------+------+-------+---------------------+--------------------+----------+------+----+------+----+
only showing top 1 row



### Join eqtl/highImpact w/ patients variants

In [26]:
eqtl_highImpact_BS.show(1)

+-----------------+--------+---------+---------+---------+------------+----------+
|unique_variant_id|   start|reference|alternate|     rsID|      symbol|chromosome|
+-----------------+--------+---------+---------+---------+------------+----------+
|  16:69571709:G:T|69571709|        G|        T|rs4783724|RP11-123C5.5|        16|
+-----------------+--------+---------+---------+---------+------------+----------+
only showing top 1 row



In [27]:
df_noNonRef.show(1)

+--------+--------+---------+---------+------+-------+---------------------+--------------------+-----------------+----------+
|   start|     end|reference|alternate|  qual|INFO_DP|splitFromMultiAllelic|           genotypes|unique_variant_id|chromosome|
+--------+--------+---------+---------+------+-------+---------------------+--------------------+-----------------+----------+
|35837713|35837714|        A|        G|271.77|     31|                 true|[{BS_0302Y3N5, 99...|  14:35837713:A:G|        14|
+--------+--------+---------+---------+------+-------+---------------------+--------------------+-----------------+----------+
only showing top 1 row



In [32]:
eqtl_highImpact_BS = eqtls_high_impact.join(df_noNonRef.drop('start','chromosome','reference','alternate'),['unique_variant_id'],'leftsemi')
print(eqtl_highImpact_BS.count())
eqtl_highImpact_BS.show(1)

242


+-----------------+--------+---------+---------+---------+------------+----------+
|unique_variant_id|   start|reference|alternate|     rsID|      symbol|chromosome|
+-----------------+--------+---------+---------+---------+------------+----------+
|  16:69571709:G:T|69571709|        G|        T|rs4783724|RP11-123C5.5|        16|
+-----------------+--------+---------+---------+---------+------------+----------+
only showing top 1 row



In [20]:
eqtl_highImpact_BS_snps = eqtl_highImpact_BS.where(((F.length("reference")==1) & (F.length("alternate") == 1)))
eqtl_highImpact_BS_indels= eqtl_highImpact_BS.where(~((F.length("reference")==1)& (F.length("alternate")== 1)))

In [21]:
eqtl_highImpact_indels_scored = eqtl_highImpact_BS_indels.join(
    cadd_ALL_indels.select('unique_variant_id','cadd_score'),['unique_variant_id'],'inner') 

In [22]:
print(eqtl_highImpact_indels_scored.count())
eqtl_highImpact_indels_scored.show(1)

21


+-----------------+--------+---------+---------+-----------+-----------+----------+----------+
|unique_variant_id|   start|reference|alternate|       rsID|     symbol|chromosome|cadd_score|
+-----------------+--------+---------+---------+-----------+-----------+----------+----------+
| 18:77097913:T:TA|77097913|        T|       TA|rs140236492|RP11-4B16.3|        18|     0.166|
+-----------------+--------+---------+---------+-----------+-----------+----------+----------+
only showing top 1 row



In [35]:
eqtls_SNPs_chrom  = eqtl_highImpact_BS_snps.where(col('chromosome') == 1)\
                                .withColumn('unique_variant_id',concat(col("chromosome"),
                                lit(":"),col("start"),lit(':'),col('reference'),lit(':'),col('alternate')))

eqtls_results_chrom = eqtls_SNPs_chrom.join(
    cadd_chrom.select('unique_variant_id','cadd_score'), ["unique_variant_id"], "inner")

eqtl_highImpact_snps_scored = eqtls_results_chrom
print(eqtl_highImpact_snps_scored.count())
eqtl_highImpact_snps_scored.show(1)

0


KeyboardInterrupt: 

In [33]:
eqtl_highImpact_scored  =  eqtl_highImpact_snps_scored.union(eqtl_highImpact_indels_scored)


NameError: name 'eqtl_highImpact_snps_scored' is not defined

In [ ]:
df_NonRef.show(1)

In [ ]:
c = ['chromosome', 'start', 'end', 'reference', 'alternate', 'unique_variant_id','qual', 
     'INFO_DP', 'splitFromMultiAllelic', 'genotypes', 'symbol', 'rsID']

In [ ]:
#df_scored = results_gt20.union(eqtl_highImpact_scored)

df_NonRef = df_NonRef.select(df_NonRef.columns+[lit('').alias("symbol"),lit('').alias("rsID"),lit('').alias("cadd_score")])

#df = df_scored.union(df_NonRef)

In [ ]:
df = df_join  
bed = whole_genome_bed

bs_name = df.withColumn('bs_id',df.genotypes.getItem(0).getItem('0')).select('bs_id').take(1)[0][0]
df = df.withColumn('calls', df.genotypes.getItem(0).getItem('6'))
df = df.withColumn('calls_str',concat(df.calls.getItem(0), lit(":"), df.calls.getItem(1) ))
df = df.withColumn('AdditiveGenotype',when(df.calls_str.contains('-')  ,-9)\
                      .otherwise(df.calls.getItem(0) + df.calls.getItem(1)))

# Create Participant ID column (should just be one participant per file)
df = df.withColumn('participant_id', df.genotypes.getItem(0).getItem('0'))
df = df.withColumn('INFO_DP',df.genotypes.getItem(0).getItem('10'))
df = df.withColumn('qual',df.genotypes.getItem(0).getItem('1'))
df = df.withColumn('PL',df.genotypes.getItem(0).getItem('9'))

select_cols = ['chromosome','start','end','participant_id','unique_variant_id','reference','alternate','calls','AdditiveGenotype','splitFromMultiAllelic','PL',  'qual','INFO_DP']
print('df columns are already filtered correctly? --',str(set(df.columns) == select_cols))
df = df.select(select_cols)  # need this?

df = df.withColumn('AdditiveGenotype',when((df.INFO_DP < 15) | (df.qual < 25) ,-9)\
                      .otherwise(df.AdditiveGenotype))


df = df.where(~ ((df.splitFromMultiAllelic == 'true') & (df.alternate == '<NON_REF>') ))
df = df.sort(['chromosome','start'])
df = df.drop(*("splitFromMultiAllelic")).dropDuplicates(['unique_variant_id','AdditiveGenotype'])

df = df.withColumn('PL_00',df.PL.getItem(0))
df = df.withColumn('PL_01',df.PL.getItem(1))
df = df.withColumn('PL_11',df.PL.getItem(2))
df = df.select(['unique_variant_id','chromosome','start','end','reference','alternate','PL','PL_00',
                               'PL_01','PL_11','AdditiveGenotype'])

df = df.withColumn('AdditiveGenotype',when((df.PL_00 == 0) & (df.PL_01 == 0) &\
                                           (col('AdditiveGenotype') == 0 ),-9).otherwise(df.AdditiveGenotype))

df = df.withColumn('AdditiveGenotype',when(((df.PL_00 == 0) & (df.PL_01 == 0) &\
                     (df.PL_11 == 0)) & (col('AdditiveGenotype') == 0 ) ,-9).otherwise(df.AdditiveGenotype))

df = df.drop(*("PL","PL_00","PL_01","PL_11"))

### Expand BED intervals

In [15]:
bed = whole_genome_bed
bed.sample(1/20000).count()
bed = bed.rdd.map(lambda x: (x['start_bed'],x['end_bed'],
                                    x['chromosome_bed'],list(range(x['start_bed'],x['end_bed']))))

#bed.select(col('_1').alias('start_bed'),col('_2').alias('end_bed'),
#                            col('_3').alias('chromosome_bed'),col('_4').alias('expanded_int'))

bed = bed.toDF(['start_bed','end_bed','chromosome_bed','expanded_int'])
print(bed.columns)

['start_bed', 'end_bed', 'chromosome_bed', 'expanded_int']


In [16]:
bed.show(1)

+---------+--------+--------------+--------------------+
|start_bed| end_bed|chromosome_bed|        expanded_int|
+---------+--------+--------------+--------------------+
| 38297023|38323217|            17|[38297023, 382970...|
+---------+--------+--------------+--------------------+
only showing top 1 row



In [22]:
bed.rdd.map(lambda x: (x['start_bed'],x['end_bed'],
                                    x['chromosome_bed'],list(range(x['start_bed'],x['end_bed']))))


PythonRDD[97] at RDD at PythonRDD.scala:53

In [ ]:
list(range(x['start_bed'],x['end_bed'])

In [26]:
b = bed.rdd.map(lambda x: (concat(x['chromosome_bed'],lit(":"))))

#b.show()


In [28]:
b.toDF()


22/06/28 15:05:06 ERROR Executor: Exception in task 0.0 in stage 33.0 (TID 172)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/pyspark/rdd.py", line 1560, in takeUpToNumLeft
    yield next(iterator)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_1503/534504526.py", line 1, in <lambda>
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 98, in lit
    return col if isinstance(col, Column) else _invoke_function("lit", col)
  File "

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 33.0 failed 1 times, most recent failure: Lost task 0.0 in stage 33.0 (TID 172) (i-0636f8f2a70597535 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/pyspark/rdd.py", line 1560, in takeUpToNumLeft
    yield next(iterator)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipyker

### Merge in cadd scored eqtl/highImpact variants to patients gvcf

In [3]:
eqtl_highImpact = spark.read.parquet('/sbgenomics/project-files/eqtls_and_high_impact.parquet')
eqtl_highImpact.show(3)

+------+---------+---------+-----------+------------+-----------------+----------+
| start|reference|alternate|       rsID|      symbol|unique_variant_id|chromosome|
+------+---------+---------+-----------+------------+-----------------+----------+
|668472|        C|        A|rs372034137|      ZNF595|     4:668472:C:A|         4|
|173022|        A|        T|  rs7678326|      ZNF718|     4:173022:A:T|         4|
|207285|        T|        C| rs28716466|CH17-262A2.1|     4:207285:T:C|         4|
+------+---------+---------+-----------+------------+-----------------+----------+
only showing top 3 rows



In [4]:
eqtl_highImpact_scored = eqtl_highImpact.withColumn('cadd_score',lit(10.0))
eqtl_highImpact_scored.show(2)

+------+---------+---------+-----------+------+-----------------+----------+----------+
| start|reference|alternate|       rsID|symbol|unique_variant_id|chromosome|cadd_score|
+------+---------+---------+-----------+------+-----------------+----------+----------+
|668472|        C|        A|rs372034137|ZNF595|     4:668472:C:A|         4|      10.0|
|173022|        A|        T|  rs7678326|ZNF718|     4:173022:A:T|         4|      10.0|
+------+---------+---------+-----------+------+-----------------+----------+----------+
only showing top 2 rows



In [17]:
eqtl_highImpact_scored.sample(1/100).show()

+---------+---------+---------+-----------+-------------+-----------------+----------+----------+
|    start|reference|alternate|       rsID|       symbol|unique_variant_id|chromosome|cadd_score|
+---------+---------+---------+-----------+-------------+-----------------+----------+----------+
| 26781023|        A|        G| rs11731852|      TBC1D19|   4:26781023:A:G|         4|      10.0|
| 61928437|        G|        T| rs79496289|   ADGRL3-AS1|   4:61928437:G:T|         4|      10.0|
| 48341676|        G|        A|  rs1876834|      SLC10A4|   4:48341676:G:A|         4|      10.0|
| 52426039|        C|        A|rs144987677|    USP46-AS1|   4:52426039:C:A|         4|      10.0|
| 57263902|        G|        A|  rs1860625|         PPAT|   4:57263902:G:A|         4|      10.0|
|163736259|        T|        G| rs62333003|        NPY5R|  4:163736259:T:G|         4|      10.0|
|  5056678|        G|        T|  rs7695179|          EVC|    4:5056678:G:T|         4|      10.0|
| 54332038|        G

In [18]:
eqtl_highImpact_scored.write.parquet('eqtl_highImpact_scored.parquet')

In [20]:
! mv eqtl_highImpact_scored.parquet/ /sbgenomics/output-files/

In [5]:
df_join = spark.read.parquet('/sbgenomics/project-files/df_bed_leftsemi.parquet')
df_noNonRef = df_join.where(~ (col('alternate') == '<NON_REF>')) # only need to get CADD scores for this df

print(df_noNonRef.count())
df_noNonRef.show(1)

2310965
+--------+--------+---------+---------+------+-------+---------------------+--------------------+-----------------+----------+
|   start|     end|reference|alternate|  qual|INFO_DP|splitFromMultiAllelic|           genotypes|unique_variant_id|chromosome|
+--------+--------+---------+---------+------+-------+---------------------+--------------------+-----------------+----------+
|35837713|35837714|        A|        G|271.77|     31|                 true|[{BS_0302Y3N5, 99...|  14:35837713:A:G|        14|
+--------+--------+---------+---------+------+-------+---------------------+--------------------+-----------------+----------+
only showing top 1 row



In [6]:
j = df_noNonRef.join(eqtl_highImpact_scored.select('unique_variant_id','rsID','symbol','cadd_score')
                     ,['unique_variant_id'],'inner')

In [7]:
j.show()

+-----------------+---------+---------+---------+---------+-------+-------+---------------------+--------------------+----------+------------+------------+----------+
|unique_variant_id|    start|      end|reference|alternate|   qual|INFO_DP|splitFromMultiAllelic|           genotypes|chromosome|        rsID|      symbol|cadd_score|
+-----------------+---------+---------+---------+---------+-------+-------+---------------------+--------------------+----------+------------+------------+----------+
|  16:69571709:G:T| 69571709| 69571710|        G|        T| 559.77|     28|                 true|[{BS_0302Y3N5, 99...|        16|   rs4783724|RP11-123C5.5|      10.0|
|   3:37364761:A:T| 37364761| 37364762|        A|        T| 427.77|     29|                 true|[{BS_0302Y3N5, 99...|         3|   rs7611351|     LRRFIP2|      10.0|
|  2:233137352:T:A|233137352|233137353|        T|        A| 852.77|     32|                 true|[{BS_0302Y3N5, 99...|         2| rs182277546|     SCARNA6|      10.0

In [16]:
j.count()

242

In [8]:
j.where(col('cadd_score').isNull()).count()

0

# Post processing: Combine all eqtl/high Impact SNPs into single dataframe

In [2]:
glob.glob('/sbgenomics/project-files/cadd*')

['/sbgenomics/project-files/cadd_chrom_files',
 '/sbgenomics/project-files/cadd_chrom_files_gt20',
 '/sbgenomics/project-files/cadd_chrom_files_parquet',
 '/sbgenomics/project-files/cadd_chrom_gt20_files',
 '/sbgenomics/project-files/cadd_gnomad_all.parquet',
 '/sbgenomics/project-files/cadd_gnomad_gt_20.tsv',
 '/sbgenomics/project-files/cadd_gt20_gnomAD_all_chroms.parquet',
 '/sbgenomics/project-files/cadd_gt20_snps.parquet']

In [ ]:
all_files = glob.glob('bs_*_eqtl_highImpct_snps.csv')
assert len(all_files) == 711

In [ ]:
li = []

# may need to use dask here, the data from each person is going to be bigger bc of +1 to start position
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)


# join this frame w/ the main cadd snp file chrom by chrom -- possibly parallelize 
# then combine with the other variants using an app. this way we only need to load the large cadd snp file one time.

# get gene names from VWB consequences table

# then search for all of these variants (the unique set) in each gvcf/parquet file,
# we need to know if its a 0,1,2 or -9. how to do this quickly?

In [ ]:
vargenemap = spark.read.parquet('/sbgenomics/project-files/tcsq_variant_gene_mappings.parquet/')
print(vargenemap.count())
vargenemap.show(3)

# Find the Additive genotype (0,1,2,-9) of all variants in each gVCF

### why duplicates in scored df?

### do all df preprossing/adding columns first, before cadd filter.

### did incorrect 'end' column get merged onto the df? just for highImpact/eQTL? --no we matched on ref/alt as well so end should always be correct

## cant just check location for genotype, what if its a different allele?

## other way to get frequencies of variants is from VWB

### make sure I'm not finding high Impact SNP scores twice (for the ones that are above 20)


### Precompute highImpact/eQTL cadd scores and then just merge them into each patients df on the unique_variant_id

In [4]:
# Load in scored variants
variants = spark.read.parquet('/sbgenomics/project-files/BS_0302Y3N5_chd_scored_variants.parquet')

variants = variants.withColumn('unique_location',concat(col("chromosome"),
                                lit(":"),col("start"),lit(":"),col("end")))
variants = variants.drop_duplicates()
print(variants.count())
variants.show(1)

177482


+--------+--------+---------+---------+-----------------+------+-------+---------------------+--------------------+------+---------+----------+------------+----------+-------------------+
|   start|     end|reference|alternate|unique_variant_id|  qual|INFO_DP|splitFromMultiAllelic|           genotypes|symbol|     rsID|cadd_score|eqtl_boolean|chromosome|    unique_location|
+--------+--------+---------+---------+-----------------+------+-------+---------------------+--------------------+------+---------+----------+------------+----------+-------------------+
|66246975|66246975|        A|        G|   1:66246975:A:G|578.77|     39|                 true|[{BS_0302Y3N5, 99...|  LEPR|rs7523912|     1.164|           1|         1|1:66246975:66246975|
+--------+--------+---------+---------+-----------------+------+-------+---------------------+--------------------+------+---------+----------+------------+----------+-------------------+
only showing top 1 row



In [7]:
variants.where(col('splitFromMultiAllelic') == 'true').count()


177482

In [3]:
variants_noneqtl = variants.where(col('eqtl_boolean') == 0)
variants_noneqtl.count()

4950

In [2]:
%%time
#df = spark.read.parquet('/sbgenomics/project-files/BS_0302Y3N5.parquet/')

# DIFFERENT gVCF than the one we got the variants from
df = spark.read.parquet('/sbgenomics/project-files/BS_03BQSV43.parquet/')



df = df.select(col('contigName').alias('chromosome'),'start','end',col('referenceAllele').alias('reference'),
         col('alternateAlleles').alias('alternate'),'qual','INFO_DP', 'splitFromMultiAllelic','genotypes')


df = df.withColumn('chromosome',F.regexp_replace(col("chromosome"), "[chr,]", "") )

#df = df.where(col('chromosome') == '15')

df = df.withColumn('alternate',F.explode('alternate'))
df = df.withColumn("start", df["start"]+1)

#df = gvcf.where( (col('alternate') == '<NON_REF>'))
df = df.where(~ (col('alternate') == '<NON_REF>'))

df = df.withColumn('unique_variant_id',concat(col("chromosome"),
                                lit(":"),col("start"),lit(':'),col('reference'),lit(':'),col('alternate')))

df = df.withColumn('unique_location',concat(col("chromosome"),
                                lit(":"),col("start"),lit(":"),col("end")))
'''
#bs_name = df.withColumn('bs_id',df.genotypes.getItem(0).getItem('0')).select('bs_id').take(1)[0][0]
df = df.withColumn('calls', df.genotypes.getItem(0).getItem('6'))
df = df.withColumn('calls_str',concat(df.calls.getItem(0), lit(":"), df.calls.getItem(1) ))
df = df.withColumn('AdditiveGenotype',when(df.calls_str.contains('-')  ,-9)\
                      .otherwise(df.calls.getItem(0) + df.calls.getItem(1)))


# Create Participant ID column (should just be one participant per file)
df = df.withColumn('participant_id', df.genotypes.getItem(0).getItem('0'))
df = df.withColumn('INFO_DP',df.genotypes.getItem(0).getItem('10'))
df = df.withColumn('qual',df.genotypes.getItem(0).getItem('1'))

df = df.withColumn('AdditiveGenotype',when((df.INFO_DP < 15) | (df.qual < 25) ,-9)\
                      .otherwise(df.AdditiveGenotype))

df = df.where(~ ((df.splitFromMultiAllelic == 'true') & (df.alternate == '<NON_REF>') ))
#df = df.sort(['chromosome','start'])
df = df.drop(*("splitFromMultiAllelic")).dropDuplicates(['unique_variant_id','AdditiveGenotype'])

#######
df = df.withColumn('PL',df.genotypes.getItem(0).getItem('9'))
df = df.withColumn('PL_00',df.PL.getItem(0))
df = df.withColumn('PL_01',df.PL.getItem(1))
df = df.withColumn('PL_11',df.PL.getItem(2))
#df = df.select(['unique_variant_id','unique_location','chromosome','start','end','reference','alternate','PL','PL_00',
#                               'PL_01','PL_11','AdditiveGenotype'])
df = df.withColumn('AdditiveGenotype',when((df.PL_00 == 0) & (df.PL_01 == 0) &\
                                           (col('AdditiveGenotype') == 0 ),-9).otherwise(df.AdditiveGenotype))
df = df.withColumn('AdditiveGenotype',when(((df.PL_00 == 0) & (df.PL_01 == 0) &\
                     (df.PL_11 == 0)) & (col('AdditiveGenotype') == 0 ) ,-9).otherwise(df.AdditiveGenotype))

df = df.drop(*("PL","PL_00","PL_01","PL_11"))
'''
df.count()

CPU times: user 67.1 ms, sys: 32.7 ms, total: 99.8 ms
Wall time: 5min 15s


166977

In [12]:
df.cache()

DataFrame[chromosome: string, start: bigint, end: bigint, reference: string, alternate: string, qual: double, INFO_DP: int, splitFromMultiAllelic: boolean, genotypes: array<struct<0:string,1:int,2:array<int>,3:array<int>,4:string,5:boolean,6:array<int>,7:int,8:string,9:array<int>,10:int>>, unique_variant_id: string, unique_location: string]

In [12]:
%%time
#df = df.drop('INFO_DP','splitFromMultiAllelic','qual','genotypes','unique_location','reference','alternate')
df.show(3)

+----------+--------+--------+---------+---------+------+-------+---------------------+--------------------+-----------------+--------------------+
|chromosome|   start|     end|reference|alternate|  qual|INFO_DP|splitFromMultiAllelic|           genotypes|unique_variant_id|     unique_location|
+----------+--------+--------+---------+---------+------+-------+---------------------+--------------------+-----------------+--------------------+
|        15|28611511|28611511|        T|       TC|610.73|     39|                 true|[{BS_03BQSV43, 99...| 15:28611511:T:TC|15:28611511:28611511|
|        15|28611518|28611518|        C|        T|   0.0|     41|                 true|[{BS_03BQSV43, 99...|  15:28611518:C:T|15:28611518:28611518|
|        15|28611525|28611525|        T|        C|643.77|     39|                 true|[{BS_03BQSV43, 99...|  15:28611525:T:C|15:28611525:28611525|
+----------+--------+--------+---------+---------+------+-------+---------------------+--------------------+----

In [28]:
#a = variants_noneqtl.rdd.mapPartitions(lambda it: [sum(1 for _ in it)])
variants_noneqtl = variants_noneqtl.repartitionByRange(4, "chromosome", "start")
#variants.rdd.mapPartitionsWithIndex(lambda x,it: [(x,sum(1 for _ in it))]).collect()

In [29]:
%%time
df_chrom = df.where(col('chromosome') == '15')
print(df_chrom.count())
chrom_locs = variants_noneqtl[variants_noneqtl['chromosome'] == '15']#['location']
print(chrom_locs.count())

166977


135
CPU times: user 53.4 ms, sys: 26.4 ms, total: 79.8 ms
Wall time: 4min 16s


In [10]:
variants.drop('INFO_DP','splitFromMultiAllelic','qual','genotypes','eqtl_boolean','unique_location').show()

+---------+---------+---------+---------+-----------------+-------------+-----------+------------------+----------+
|    start|      end|reference|alternate|unique_variant_id|       symbol|       rsID|        cadd_score|chromosome|
+---------+---------+---------+---------+-----------------+-------------+-----------+------------------+----------+
| 66246975| 66246975|        A|        G|   1:66246975:A:G|         LEPR|  rs7523912|             1.164|         1|
| 61817832| 61817832|        A|        C|   1:61817832:A:C|RP11-430G17.3|  rs6700692|             4.789|         1|
| 77283842| 77283842|        C|        T|   1:77283842:C:T|          AK5|  rs1779173|             4.533|         1|
|150989647|150989647|        C|        T|  1:150989647:C:T|       ZNF687|  rs4590677|             0.211|         1|
|154975851|154975851|        G|        T|  1:154975851:G:T|        DCST2| rs12087657|6.1610000000000005|         1|
| 76102821| 76102821|        T|        G|   1:76102821:T:G|RP11-510C10.2

In [30]:
chrom_locs_trim = chrom_locs.select(col('chromosome').alias('var_chrom'),col('start').alias('var_start'),
                                                col('unique_variant_id').alias('var_unique_variant_id'),
                                                col('end').alias('var_end'))
chrom_locs_trim.count()

135

In [31]:
%%time
df_join = df_chrom.join(chrom_locs_trim, 
                  (df_chrom['chromosome'] == chrom_locs_trim['var_chrom']) &\
                  (df_chrom['start'] <= chrom_locs_trim['var_start']) &\
                  (df_chrom['end'] >= chrom_locs_trim['var_start']),'inner' )

df_join.drop().show()

+----------+--------+--------+-----------------+---------+---------+---------------------+--------+
|chromosome|   start|     end|unique_variant_id|var_chrom|var_start|var_unique_variant_id| var_end|
+----------+--------+--------+-----------------+---------+---------+---------------------+--------+
|        15|34528068|34528068|  15:34528067:G:A|       15| 34528068|      15:34528068:G:A|34528068|
|        15|37019897|37019897|  15:37019896:A:G|       15| 37019897|      15:37019897:A:G|37019897|
|        15|39589977|39589977|  15:39589976:A:G|       15| 39589977|      15:39589977:A:G|39589977|
|        15|39973598|39973598|  15:39973597:A:G|       15| 39973598|      15:39973598:A:G|39973598|
|        15|40459356|40459356|  15:40459355:C:A|       15| 40459356|      15:40459356:C:A|40459356|
|        15|40813728|40813728|  15:40813727:T:G|       15| 40813728|      15:40813728:T:G|40813728|
|        15|41573327|41573327|  15:41573326:G:T|       15| 41573327|      15:41573327:G:T|41573327|


In [32]:
df_join.count()

50

In [ ]:
+----------+--------+--------+---------+---------+-----------------+--------------------+---------+---------+---------------------+--------+
|chromosome|   start|     end|reference|alternate|unique_variant_id|     unique_location|var_chrom|var_start|var_unique_variant_id| var_end|
+----------+--------+--------+---------+---------+-----------------+--------------------+---------+---------+---------------------+--------+
|        15|34528067|34528068|        G|        A|  15:34528067:G:A|15:34528067:34528068|       15| 34528068|      15:34528068:G:A|34528068|
|        15|37019896|37019897|        A|        G|  15:37019896:A:G|15:37019896:37019897|       15| 37019897|      15:37019897:A:G|37019897|
|        15|39589976|39589977|        A|        G|  15:39589976:A:G|15:39589976:39589977|       15| 39589977|      15:39589977:A:G|39589977|
|        15|39973597|39973598|        A|        G|  15:39973597:A:G|15:39973597:39973598|       15| 39973598|      15:39973598:A:G|39973598|
|        15|40459355|40459356|        C|        A|  15:40459355:C:A|15:40459355:40459356|       15| 40459356|      15:40459356:C:A|40459356|
|        15|40813727|40813728|        T|        G|  15:40813727:T:G|15:40813727:40813728|       15| 40813728|      15:40813728:T:G|40813728|

### Join on unique location to find set of variants we have direct genotypes for 
##### Can filter out NON_REF rows when searching initially ,----- actually cannot do this.

In [5]:
%%time
# first get set of variants/locs from the input
uniq_locs = [row['unique_location'] for row in variants_noneqtl.where(col('chromosome') == '15')\
                                                                .select('unique_location').collect()]
print(len(uniq_locs))

135
CPU times: user 9.89 ms, sys: 0 ns, total: 9.89 ms
Wall time: 1.72 s


In [6]:
%%time
vars_found = variants_noneqtl.where(col('chromosome') == '15')\
                .join(df.where(col('chromosome') == '15')\
                      .select('unique_location','unique_variant_id','AdditiveGenotype')\
                      ,['unique_location'],'inner')
print(vars_found.count())
#vars_found.drop('splitFromMultiAllelic','genotypes').show(5)

1
CPU times: user 147 ms, sys: 47.6 ms, total: 194 ms
Wall time: 18min 54s


In [12]:
print(df.where(col('chromosome') == '15').count())
print(variants_noneqtl.where(col('chromosome') == '15').count())

KeyboardInterrupt: 

In [7]:
#set(a.select('unique_location').collect())

# what if someone has a different variation, still just 1 or 2?
uniq_locs = [row['unique_location'] for row in vars_found.select('unique_location').collect()]
uniq_locs

['15:22851829:22851830']

In [ ]:
# Filter out all eqtl variants first and find the unique set of locations/variants for the entire cohort.
# Then, merge this set w/ a gvcf to see which vars we already have a genotype match for. set these off to theside.
# Then do anti merge do get list of vars that we dont have direct genotype matches for

In [18]:
len(uniq_locs)

1

In [9]:
# get the list of locations we dont have direct genotype hits for
vars_missing = variants_noneqtl.where(col('chromosome') == '15').where(~col('unique_location').isin(uniq_locs))
vars_missing.count()

129

In [10]:
missing_uniq_locs = [row['unique_location'] for row in vars_missing.select('unique_location').collect()]


In [11]:
m  =missing_uniq_locs[0]
m

'15:60764664:60764664'

In [12]:
m.split(':')

['15', '60764664', '60764664']

In [13]:
df.show(10)

KeyboardInterrupt: 

In [17]:
df_missing = pd.DataFrame([i.split(':') for i in missing_uniq_locs],columns=['chrom','start','end'])
df_missing['location'] = missing_uniq_locs

In [18]:
%%time
df_chrom = df.where(col('chromosome') == '15')
chrom_locs = df_missing[df_missing['chrom'] == '15']['location']

CPU times: user 3.8 ms, sys: 0 ns, total: 3.8 ms
Wall time: 17.7 ms


In [19]:
df_missing.head(5)

,chrom,start,end,location
0,15,60764664,60764664,15:60764664:60764664
1,15,32665918,32665918,15:32665918:32665918
2,15,83970598,83970598,15:83970598:83970598
3,15,39973598,39973598,15:39973598:39973598
4,15,66296165,66296165,15:66296165:66296165


# Examine results of getting genotypes

In [3]:
df = spark.read.parquet('/sbgenomics/project-files/BS_0302Y3N5.parquet/')

df = df.select(col('contigName').alias('chromosome'),'start','end',col('referenceAllele').alias('reference'),
                        col('alternateAlleles').alias('alternate'),'qual','INFO_DP', 'splitFromMultiAllelic',
                                'genotypes').withColumn('alternate',F.explode('alternate')).withColumn('chromosome',F.regexp_replace(col("chromosome"), "[chr,]", "") )                    
df = df.withColumn("start", df["start"]+1) #df_count = df.count() 

df = df.withColumn('unique_variant_id',concat(col("chromosome"), lit(":"),
                    col("start"),lit(':'),col('reference'),lit(':'),col('alternate'))) 

df = df.where(col('chromosome') == '15')

df.count()

11927164

In [4]:
df = df.where(~ (col('alternate') == '<NON_REF>'))

In [16]:
df.count()

169129

In [11]:
b = spark.read.parquet('/sbgenomics/project-files/bed_protein_coding_coords_only.parquet')
print(b.rdd.getNumPartitions())
b.rdd.mapPartitions(lambda it: [sum(1 for _ in it)]).collect()

2


[37888, 2062]

In [12]:
e = spark.read.parquet('/sbgenomics/project-files/BS_0302Y3N5_chd_scored_variants.parquet')
print(e.rdd.getNumPartitions())
e.rdd.mapPartitions(lambda it: [sum(1 for _ in it)]).collect()

2


[151775, 25707]

In [15]:
e.where(col('eqtl_boolean') == 0).coalesce(1).rdd.mapPartitions(lambda it: [sum(1 for _ in it)]).collect()

[4950]

In [17]:
d = df.rdd.mapPartitions(lambda it: [sum(1 for _ in it)]).collect()
d

In [20]:
snps_gt20 = spark.read.parquet('/sbgenomics/project-files/cadd_gt20_snps.parquet')
print(snps_gt20.count())
snps_gt20_dedup = snps_gt20.dropDuplicates()
snps_gt20_dedup.count()

175854962


87927481

In [21]:
snps_gt20_dedup.show(1)

+---------+---------+---------+----------+-----------------+----------+
|    start|reference|alternate|cadd_score|unique_variant_id|chromosome|
+---------+---------+---------+----------+-----------------+----------+
|112769487|        A|        C|      20.0| 12:112769487:A:C|        12|
+---------+---------+---------+----------+-----------------+----------+
only showing top 1 row



In [22]:
2*87927481

175854962

In [23]:
snps_gt20_dedup.repartitionByRange(60, "chromosome").write\
      .mode('overwrite')\
      .partitionBy("chromosome")\
      .parquet("cadd_gt20_snps_2.parquet")

In [24]:
!mv cadd_gt20_snps_2.parquet/ /sbgenomics/output-files/

In [10]:
t = spark.read.parquet('/sbgenomics/project-files/_8_BS_0302Y3N5_chd_scored_variants.parquet')
print(t.count())

22/07/20 12:27:08 WARN DataSource: All paths were ignored:
  file:/sbgenomics/project-files/_8_BS_0302Y3N5_chd_scored_variants.parquet


177482


In [4]:
t.drop_duplicates().count()

177482

In [4]:
t.show(1)

+---------+---------+---------+---------+-----------------+-------+-------+---------------------+--------------------+------+----+----------+------------+----------+
|    start|      end|reference|alternate|unique_variant_id|   qual|INFO_DP|splitFromMultiAllelic|           genotypes|symbol|rsID|cadd_score|eqtl_boolean|chromosome|
+---------+---------+---------+---------+-----------------+-------+-------+---------------------+--------------------+------+----+----------+------------+----------+
|125889890|125889890|        A|        G| 10:125889890:A:G|1019.77|     70|                 true|[{BS_0302Y3N5, 99...|      |    |      20.7|           0|        10|
+---------+---------+---------+---------+-----------------+-------+-------+---------------------+--------------------+------+----+----------+------------+----------+
only showing top 1 row



In [7]:
t.where(col('cadd_score') < 20).count()

172660

In [11]:
#Counter(list(t.where(col('eqtl_boolean') == 1).select(col('cadd_score')).toPandas()['cadd_score'])).most_common()

In [6]:
t15 = t.where((col('unique_variant_id').startswith('15')))# & (col('AdditiveGenotype') == -9))
print(t15.count())
t15.show(2)

6920
+--------+--------+---------+---------+-----------------+------+-------+---------------------+--------------------+------------+-----------+-------------------+------------+----------+
|   start|     end|reference|alternate|unique_variant_id|  qual|INFO_DP|splitFromMultiAllelic|           genotypes|      symbol|       rsID|         cadd_score|eqtl_boolean|chromosome|
+--------+--------+---------+---------+-----------------+------+-------+---------------------+--------------------+------------+-----------+-------------------+------------+----------+
|57780703|57780703|        A|        C|  15:57780703:A:C|   0.0|     38|                 true|[{BS_0302Y3N5, 80...|            |           |               20.1|           0|        15|
|78773847|78773847|        A|        G|  15:78773847:A:G|382.77|     28|                 true|[{BS_0302Y3N5, 99...|RP11-285A1.1|rs181072044|0.16699999999999998|           1|        15|
+--------+--------+---------+---------+-----------------+------+------

In [7]:
t15 = t.where(col('chromosome') == '15').select('unique_variant_id')

In [8]:
t15v = list(t15.select('unique_variant_id').toPandas()['unique_variant_id']) 
print(len(t15v))
len(set(t15v))

6920


5760

In [9]:
df15 = list(df.select('unique_variant_id').toPandas()['unique_variant_id']) 
print(len(df15))
len(set(df15))

169129


169129

In [10]:
len(set(t15v) - set(df15))

0

In [11]:
len(set(df15).intersection(set(t15v)))

5760

In [12]:
df15

['15:28611511:T:TC',
 '15:28611518:C:T',
 '15:28611525:T:C',
 '15:28612776:T:G',
 '15:28612777:T:G',
 '15:28612778:T:G',
 '15:28613520:T:G',
 '15:28614975:C:T',
 '15:28615086:C:T',
 '15:28615277:G:C',
 '15:28615339:G:A',
 '15:28619609:CTTTTTTTT:C',
 '15:28623163:A:G',
 '15:28623172:T:G',
 '15:28623182:A:G',
 '15:28623537:T:C',
 '15:28625642:C:CT',
 '15:28628095:A:G',
 '15:28628155:T:C',
 '15:28628192:G:T',
 '15:28629273:G:C',
 '15:28634773:T:C',
 '15:28634779:C:T',
 '15:28636563:C:T',
 '15:28637107:G:C',
 '15:28637113:G:C',
 '15:28645213:C:T',
 '15:28646616:T:C',
 '15:28646917:GTC:G',
 '15:28646917:G:GTC',
 '15:28646917:G:GTCTGTC',
 '15:28646921:C:G',
 '15:28646935:A:C',
 '15:28658997:T:C',
 '15:28659128:G:A',
 '15:28659526:A:T',
 '15:28659775:G:A',
 '15:28659781:G:T',
 '15:28662903:AT:A',
 '15:28663630:A:G',
 '15:28665162:CT:C',
 '15:28668999:G:A',
 '15:28669394:CT:C',
 '15:28669394:C:CT',
 '15:28669394:C:CTT',
 '15:28670474:G:C',
 '15:28671042:C:T',
 '15:28672671:G:A',
 '15:28673044:

In [14]:
Counter([len(i.split(':')) for i in t15v])

Counter({4: 6920})

In [16]:
df.show(1)

+----------+--------+--------+---------+---------+------+-------+---------------------+--------------------+-----------------+--------------------+
|chromosome|   start|     end|reference|alternate|  qual|INFO_DP|splitFromMultiAllelic|           genotypes|unique_variant_id|     unique_location|
+----------+--------+--------+---------+---------+------+-------+---------------------+--------------------+-----------------+--------------------+
|        15|28611511|28611511|        T|       TC|610.73|     39|                 true|[{BS_03BQSV43, 99...| 15:28611511:T:TC|15:28611511:28611511|
+----------+--------+--------+---------+---------+------+-------+---------------------+--------------------+-----------------+--------------------+
only showing top 1 row



In [17]:
set(t15v).issubset(set(df15))

False

In [20]:
df.where(col('unique_variant_id') == '15:100603213:A:T' ).show()

+----------+-----+---+---------+---------+----+-------+---------------------+---------+-----------------+---------------+
|chromosome|start|end|reference|alternate|qual|INFO_DP|splitFromMultiAllelic|genotypes|unique_variant_id|unique_location|
+----------+-----+---+---------+---------+----+-------+---------------------+---------+-----------------+---------------+
+----------+-----+---+---------+---------+----+-------+---------------------+---------+-----------------+---------------+



In [22]:
df.select('unique_variant_id').show()

+--------------------+
|   unique_variant_id|
+--------------------+
|    15:28611511:T:TC|
|     15:28611518:C:T|
|     15:28611525:T:C|
|     15:28612776:T:G|
|     15:28612777:T:G|
|     15:28612778:T:G|
|     15:28613520:T:G|
|     15:28614975:C:T|
|     15:28615086:C:T|
|     15:28615277:G:C|
|     15:28615339:G:A|
|15:28619609:CTTTT...|
|     15:28623163:A:G|
|     15:28623172:T:G|
|     15:28623182:A:G|
|     15:28623537:T:C|
|    15:28625642:C:CT|
|     15:28628095:A:G|
|     15:28628155:T:C|
|     15:28628192:G:T|
+--------------------+
only showing top 20 rows



In [4]:
t.drop('qual','INFO_DP','var_unique_variant_id','genotypes',
       'unique_variant_id','unique_location').show(10) # 'reference','alternate'


+--------+--------+---------+---------+---------------------+---------+---------+--------+----------+
|   start|     end|reference|alternate|splitFromMultiAllelic|var_chrom|var_start| var_end|chromosome|
+--------+--------+---------+---------+---------------------+---------+---------+--------+----------+
|30922706|30922706|        G|        T|                 true|        6| 30922706|30922706|         6|
|30922706|30922706|        G|<NON_REF>|                 true|        6| 30922706|30922706|         6|
|30925350|30925350|        G|        A|                 true|        6| 30925350|30925350|         6|
|30925350|30925350|        G|<NON_REF>|                 true|        6| 30925350|30925350|         6|
|31138114|31138114|        A|        G|                 true|        6| 31138114|31138114|         6|
|31138114|31138114|        A|<NON_REF>|                 true|        6| 31138114|31138114|         6|
|31144707|31144707|        C|        T|                 true|        6| 31144707|3

In [ ]:
startsWith

In [ ]:
#t.where(col('splitFromMultiAllelic') == 'true').count()
#t.where(col('start') == col('end')).count()

In [4]:
%%time
df = t
df = df.withColumn('calls', df.genotypes.getItem(0).getItem('6'))
df = df.withColumn('calls_str',concat(df.calls.getItem(0), lit(":"), df.calls.getItem(1) ))
df = df.withColumn('AdditiveGenotype',when(df.calls_str.contains('-')  ,-9)\
                      .otherwise(df.calls.getItem(0) + df.calls.getItem(1)))

# Create Participant ID column (should just be one participant per file)
df = df.withColumn('participant_id', df.genotypes.getItem(0).getItem('0'))
df = df.withColumn('INFO_DP',df.genotypes.getItem(0).getItem('10'))
df = df.withColumn('qual',df.genotypes.getItem(0).getItem('1'))
df = df.withColumn('PL',df.genotypes.getItem(0).getItem('9'))

select_cols = ['chromosome','start','end','participant_id','unique_variant_id','unique_location','reference','alternate','calls','AdditiveGenotype','splitFromMultiAllelic','PL',  'qual','INFO_DP']

#print('df columns are already filtered correctly? --',str(set(df.columns) == select_cols))
#print(df.columns); time.sleep(10)  #df = df.select(select_cols)  # need this?

df = df.withColumn('AdditiveGenotype',when((df.INFO_DP < 15) | (df.qual < 25) ,-9)\
                      .otherwise(df.AdditiveGenotype))
df = df.where(~ ((df.splitFromMultiAllelic == 'true') & (df.alternate == '<NON_REF>') ))

#df = df.sort(['chromosome','start'])
#df = df.drop(*("splitFromMultiAllelic")).dropDuplicates(['unique_variant_id','AdditiveGenotype'])

df = df.withColumn('PL_00',df.PL.getItem(0)).withColumn('PL_01',df.PL.getItem(1)).withColumn('PL_11',df.PL.getItem(2))
#df = df.select(['unique_variant_id','unique_location','chromosome','start','end','reference','alternate','PL','PL_00', 'PL_01','PL_11','AdditiveGenotype'])

df = df.withColumn('AdditiveGenotype',when((df.PL_00 == 0) & (df.PL_01 == 0) &\
                                           (col('AdditiveGenotype') == 0 ),-9).otherwise(df.AdditiveGenotype))
df = df.withColumn('AdditiveGenotype',when(((df.PL_00 == 0) & (df.PL_01 == 0) &\
                     (df.PL_11 == 0)) & (col('AdditiveGenotype') == 0 ) ,-9).otherwise(df.AdditiveGenotype))

df = df.drop(*("PL","PL_00","PL_01","PL_11"))


CPU times: user 22.8 ms, sys: 3.87 ms, total: 26.6 ms
Wall time: 560 ms


In [7]:
print(df.count())
df.show(1)

5038
+--------+--------+---------+---------+----+-------+---------------------+--------------------+-----------------+-------------------+---------+---------+---------------------+--------+----------+------+---------+----------------+--------------+
|   start|     end|reference|alternate|qual|INFO_DP|splitFromMultiAllelic|           genotypes|unique_variant_id|    unique_location|var_chrom|var_start|var_unique_variant_id| var_end|chromosome| calls|calls_str|AdditiveGenotype|participant_id|
+--------+--------+---------+---------+----+-------+---------------------+--------------------+-----------------+-------------------+---------+---------+---------------------+--------+----------+------+---------+----------------+--------------+
|30922706|30922706|        G|        T|  99|     40|                 true|[{BS_0302Y3N5, 99...|   6:30922706:G:T|6:30922706:30922706|        6| 30922706|       6:30922706:G:T|30922706|         6|[0, 1]|      0:1|               1|   BS_0302Y3N5|
+--------+-----

In [8]:
df = df.select(['chromosome','start','end','unique_location','unique_variant_id','AdditiveGenotype'])
df.show()

+----------+--------+--------+-------------------+-----------------+----------------+
|chromosome|   start|     end|    unique_location|unique_variant_id|AdditiveGenotype|
+----------+--------+--------+-------------------+-----------------+----------------+
|         6|30922706|30922706|6:30922706:30922706|   6:30922706:G:T|               1|
|         6|30925350|30925350|6:30925350:30925350|   6:30925350:G:A|               1|
|         6|31138114|31138114|6:31138114:31138114|   6:31138114:A:G|               2|
|         6|31144707|31144707|6:31144707:31144707|   6:31144707:C:T|               1|
|         6|31144960|31144960|6:31144960:31144960|   6:31144960:C:A|               1|
|         6|31148407|31148407|6:31148407:31148407|   6:31148407:G:T|               0|
|         6|31151121|31151121|6:31151121:31151121|   6:31151121:A:T|               1|
|         6|31154538|31154538|6:31154538:31154538|   6:31154538:G:C|               1|
|         6|31154705|31154705|6:31154705:31154705|   6

# Merge genes into the results of the new genotypes dataframe

In [5]:
t = spark.read.parquet('/sbgenomics/project-files/_1_test_FULL_join_july.parquet')
print(t.count())
t.show(1)

22/07/13 13:07:49 WARN DataSource: All paths were ignored:
  file:/sbgenomics/project-files/_1_test_FULL_join_july.parquet


10021
+--------+--------+---------+---------+------+-------+---------------------+--------------------+-----------------+-------------------+---------+---------+---------------------+--------+----------+
|   start|     end|reference|alternate|  qual|INFO_DP|splitFromMultiAllelic|           genotypes|unique_variant_id|    unique_location|var_chrom|var_start|var_unique_variant_id| var_end|chromosome|
+--------+--------+---------+---------+------+-------+---------------------+--------------------+-----------------+-------------------+---------+---------+---------------------+--------+----------+
|30922706|30922706|        G|        T|581.77|     40|                 true|[{BS_0302Y3N5, 99...|   6:30922706:G:T|6:30922706:30922706|        6| 30922706|       6:30922706:G:T|30922706|         6|
+--------+--------+---------+---------+------+-------+---------------------+--------------------+-----------------+-------------------+---------+---------+---------------------+--------+----------+
only

In [4]:
gene_map = spark.read.option("mode", "DROPMALFORMED").parquet('/sbgenomics/project-files/tcsq_variant_gene_mappings_july4_3.parquet')
gene_map = gene_map.withColumn('unique_variant_id',concat(col("chromosome"), lit(":"),
                                                   col("start"),lit(':'),col('reference'),
                                                   lit(':'),col('alternate')))
print(gene_map.count())
gene_map.show(3)

10932230
+----------+--------+---------+---------+---------+---------+-----------------+
|chromosome|   start|reference|alternate|     rsID|   symbol|unique_variant_id|
+----------+--------+---------+---------+---------+---------+-----------------+
|         6|31674322|        G|        A|     null|   LY6G5B|   6:31674322:G:A|
|         6|25727870|        C|        T|rs9356985|HIST1H2BA|   6:25727870:C:T|
|         6|25957982|       AT|        A|     null|   TRIM38|  6:25957982:AT:A|
+----------+--------+---------+---------+---------+---------+-----------------+
only showing top 3 rows



In [8]:
mapped_df = t.select(['unique_variant_id','genotypes']).join(gene_map.select('unique_variant_id','symbol','rsID'),'unique_variant_id','inner')
print(mapped_df.count())
mapped_df.show()

1653


+-----------------+--------------------+----------+-----------+
|unique_variant_id|           genotypes|    symbol|       rsID|
+-----------------+--------------------+----------+-----------+
|   6:32828908:A:G|[{BS_0302Y3N5, 99...|      TAP2|   rs241448|
|   6:32828974:T:C|[{BS_0302Y3N5, 99...|      TAP2|   rs241447|
|   6:31636814:C:T|[{BS_0302Y3N5, 99...|   MIR6832|    rs10885|
|   6:32828974:T:C|[{BS_0302Y3N5, 99...|AL669918.1|   rs241447|
|   6:32521954:G:C|[{BS_0302Y3N5, 99...|  HLA-DRB5| rs41554620|
|   6:32519432:C:G|[{BS_0302Y3N5, 21...|  HLA-DRB5|rs189435670|
|   6:32332045:G:A|[{BS_0302Y3N5, 99...|   C6orf10|  rs1003878|
|   6:31356822:T:G|[{BS_0302Y3N5, 6,...|   MIR6891|  rs1050538|
|   6:33069222:T:C|[{BS_0302Y3N5, 99...|  HLA-DPA1|  rs1042190|
|   6:29173855:T:C|[{BS_0302Y3N5, 99...|     OR2J2|  rs3116855|
|   6:35317943:A:C|[{BS_0302Y3N5, 99...|      DEF6|  rs2395617|
|   6:32519567:T:C|[{BS_0302Y3N5, 99...|  HLA-DRB5|rs201863832|
|   6:29101487:T:C|[{BS_0302Y3N5, 99...|

# Normalize by length

In [5]:

gene_lens = spark.read.option('sep','\t').option('header',True).csv('/sbgenomics/project-files/gene_regions_refseq')
print(gene_lens.count())
gene_lens.show(1)

172809
+----+--------------+-----+------+--------+--------+--------+--------+---------+--------------------+--------------------+-----+--------+------------+----------+--------------------+
|#bin|          name|chrom|strand| txStart|   txEnd|cdsStart|  cdsEnd|exonCount|          exonStarts|            exonEnds|score|   name2|cdsStartStat|cdsEndStat|          exonFrames|
+----+--------------+-----+------+--------+--------+--------+--------+---------+--------------------+--------------------+-----+--------+------------+----------+--------------------+
|   0|NM_001276352.2| chr1|     -|67092164|67134970|67093579|67127240|        9|67092164,67096251...|67093604,67096321...|    0|C1orf141|        cmpl|      cmpl|2,1,0,1,2,0,0,-1,-1,|
+----+--------------+-----+------+--------+--------+--------+--------+---------+--------------------+--------------------+-----+--------+------------+----------+--------------------+
only showing top 1 row



In [26]:
%%time
chroms = ['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '22', '3', '4', '5', '6', '7','8', '9', 'X','Y']

AdditiveGenotypes = []

for CHROM in ['15']:#chroms:
    #df_chrom = df.where(col('chromosome') == CHROM)
    #chrom_locs = df_missing[df_missing['chrom'] == CHROM]['location']
    
    for LOC in chrom_locs:
        start_loc = int(LOC.split(':')[1])
        end_loc = int(LOC.split(':')[2])
        
        print(f'Searching for {start_loc}')
        FOUND = False
        N = 1
    
        while FOUND == False:
            print(f'\tSearch: {start_loc - N}...',end='')
            df_loc = df_chrom.where(col('start') ==  start_loc - N)
            
            if df_loc.count() == 1:
                FOUND = True
                print('Found!')
                end_pos_df = [row['end'] for row in df_loc.select('end').collect()] 
                
                if end_loc <= end_pos_df: 
                    AdditiveGenotypes.append((LOC,[row['AdditiveGenotype'] for row in\
                                                          df_loc.select('AdditiveGenotype').collect()]) )
                else:
                    AdditiveGenotypes.append((LOC, -9 )) # not reported in gVCF, assign missing (-9)
            elif df_loc.count() == 0:
                print('Not Found.')
                N=N+1
            else:
                print('Found Loc with more than one match!!!')
                break
    

Searching for 60764664
	Search: 60764663...

KeyboardInterrupt: 

Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


In [41]:

# Returns index of x in arr if present, else -1
def binary_search(arr, low, high, x):
 
    # Check base case
    if high >= low:
 
        mid = (high + low) // 2
 
        # If element is present at the middle itself
        if arr[mid] == x:
            return mid
 
        # If element is smaller than mid, then it can only
        # be present in left subarray
        elif arr[mid] > x:
            return binary_search(arr, low, mid - 1, x)
 
        # Else the element can only be present in right subarray
        else:
            return binary_search(arr, mid + 1, high, x)
 
    else:
        # Element is not present in the array
        return -1
 
# Test array
arr = [ 2, 3, 4, 10, 40 ]
x = 10
 
# Function call
result = binary_search(arr, 0, len(arr)-1, x)
 
if result != -1:
    print("Element is present at index", str(result))
else:
    print("Element is not present in array")

Element is present at index 3
